# TensorFlow GNN  药物心脏毒性检测
Introduction to TensorFlow GNN: a Graph Attention baseline for drug cardiotoxicity detection  
![](gnn1.webp)

图表显然是一个非常通用和强大的概念，可以用来表示许多不同类型的数据。例如：
1. 社交网络可以被认为是一个图表，其中节点对应于不同的用户，边缘对应于他们的关系（即“友谊”、“追随者”等）。  
2. 一个国家可以表示为图表，其中城市被认为是节点，连接它们的道路扮演边缘的角色。

图形神经网络（GNN）是我们能够将神经网络应用于图形结构数据以学习对其进行预测的一种方式。它们的架构通常涉及堆叠消息传递层，每个层都会更新每个节点的功能i在图表中，通过将一些函数应用于其邻居的特征，
<img src="gnn2.webp" width="400"/>
GNN 应用的一个典型例子是图分类问题，我们的任务是在一组有限的可能选项中预测给定图的类别。 在监督设置中，我们有一组标记图，可以从中学习参数 {θV,θE} 通过最小化我们的预测的分类交叉熵损失来优化 GNN。 这些是在对所有节点应用最终聚合或池化操作后从 GNN 获得的，这又应该是无序的，以保留输出在节点重新标记下不变的重要属性。

## 依赖库的安装和导入

由于 TF-GNN 目前处于早期 alpha 发布阶段，因此在 Kaggle 笔记本环境中安装会出现一些问题。我发现以下组合效果很好：

In [ ]:
from IPython.display import clear_output

# install non-Python dependencies
!apt-get -y install graphviz graphviz-dev

# Upgrade to TensorFlow 2.8
!pip install tensorflow==2.8 tensorflow-io==0.25.0 tfds-nightly pygraphviz

# Install TensorFlow-GNN
!pip install tensorflow_gnn==0.2.0

# Fix some dependencies
!pip install httplib2==0.20.4

clear_output()

现在我们已经拥有了继续操作所需的一切

In [1]:
# import pygraphviz as pgv
from tqdm import tqdm
from IPython.display import Image

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import tensorflow_gnn as tfgnn
import tensorflow_datasets as tfds

from tensorflow_gnn import runner
from tensorflow_gnn.models import gat_v2

print(f'Using TensorFlow v{tf.__version__} and TensorFlow-GNN v{tfgnn.__version__}')
print(f'GPUs available: {tf.config.list_physical_devices("GPU")}')

/home/heng/anaconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using TensorFlow v2.8.0 and TensorFlow-GNN v0.2.0
GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
tf.__version__

'2.8.0'

## 数据预处理

Unlike other types of data, there is no standard encoding for graphs. Indeed, depending on the intended application the graph structure can be represented by:

1. An adjacency matrix  Aij specifying whether the edge  i→j is present or not in  E .  
1. An adjacency list  Ni for each node  i∈V , specifying all the nodes  j∈V for which an edge  (i,j)∈E  
1. A list of edges  (i,j)∈E

为了解决这个问题，TF-GNN 引入了 GraphTensor 对象，它封装了图结构以及节点、边和图本身的特征。 这些对象遵循图形模式，该模式指定节点和边的类型以及图形中应出现的所有功能。 因此，任何 TF-GNN 训练流程的第一步都应该是将输入数据从给定的格式转换为 GraphTensor 格式。 然后，这些 GraphTensor 对象可以像 tf.Tensor 一样被我们的 TF-GNN 模型批量处理和使用，从而极大地简化了我们的工作过程。

本节的目标是执行上述任务，为我们的数据生成 DatasetProvider 对象（有关 DatasetProvider 协议的说明，请参阅此处和下一节）。 然后让我们看一下 CardioTox 数据集，我们可以从 TensorFlow 数据集 (TF-DS) 自动下载该数据集：

In [4]:
dataset_splits, dataset_info = tfds.load('cardiotox', data_dir='data/tfds', with_info=True)


print(dataset_info.description)

Drug Cardiotoxicity dataset [1-2] is a molecule classification task to detect
cardiotoxicity caused by binding hERG target, a protein associated with heart
beat rhythm. The data covers over 9000 molecules with hERG activity.

Note:

1. The data is split into four splits: train, test-iid, test-ood1, test-ood2.

2. Each molecule in the dataset has 2D graph annotations which is designed to
facilitate graph neural network modeling. Nodes are the atoms of the molecule
and edges are the bonds. Each atom is represented as a vector encoding basic
atom information such as atom type. Similar logic applies to bonds.

3. We include Tanimoto fingerprint distance (to training data) for each molecule
in the test sets to facilitate research on distributional shift in graph domain.

For each example, the features include:
  atoms: a 2D tensor with shape (60, 27) storing node features. Molecules with
    less than 60 atoms are padded with zeros. Each atom has 27 atom features.
  pairs: a 3D tensor with 

如前所述，我们只有一组节点（我们称之为“原子”）和一组边（我们称之为“键”）。 当然，所有边的两个端点都有“原子”型节点。 节点和边都有一个特征向量，前者是 27 维“atom_features”向量，后者是 12 维“bond_features”向量。 此外，图本身具有给出其上下文的全局特征，在本例中是毒性类别“毒性”，这实际上是我们想要预测的标签，以及我们通常会忽略的分子 ID“分子 ID”。

上述所有内容都可以编码在以下图形模式中，指定图形的结构和内容：

In [5]:
graph_schema_pbtxt = """
node_sets {
  key: "atom"
  value {
    description: "An atom in the molecule."

    features {
      key: "atom_features"
      value: {
        description: "[DATA] The features of the atom."
        dtype: DT_FLOAT
        shape { dim { size: 27 } }
      }
    }
  }
}

edge_sets {
  key: "bond"
  value {
    description: "A bond between two atoms in the molecule."
    source: "atom"
    target: "atom"

    features {
      key: "bond_features"
      value: {
        description: "[DATA] The features of the bond."
        dtype: DT_FLOAT
        shape { dim { size: 12 } }
      }
    }
  }
}

context {
  features {
    key: "toxicity"
    value: {
      description: "[LABEL] The toxicity class of the molecule (0 -> non-toxic; 1 -> toxic)."
      dtype: DT_INT64
    }
  }
  
  features {
    key: "molecule_id"
    value: {
      description: "[LABEL] The id of the molecule."
      dtype: DT_STRING
    }
  }
}
"""

这个 schema 是一个文本 protobuf，我们可以解析它以获得 GraphTensorSpec（想想 GraphTensor 对象的 TensorSpec）：

In [6]:
graph_schema = tfgnn.parse_schema(graph_schema_pbtxt)
graph_spec = tfgnn.create_graph_spec_from_schema_pb(graph_schema)

然后，我们应该将输入数据集转换为符合 graph_spec 的 GraphTensor 对象，我们将使用以下辅助函数来完成此操作：

In [7]:
def make_graph_tensor(datapoint):
    """
    Convert a datapoint from the TF-DS CardioTox dataset into a `GraphTensor`.
    """
    # atom_mask is non-zero only for real atoms
    # [ V, ]
    atom_indices = tf.squeeze(tf.where(datapoint['atom_mask']), axis=1)
    
    # only keep features of real atoms
    # [ V, 27 ]
    atom_features = tf.gather(datapoint['atoms'], atom_indices)
    
    # restrict the bond mask to real atoms
    # [ V, V ]
    pair_mask = tf.gather(tf.gather(datapoint['pair_mask'], atom_indices, axis=0), atom_indices, axis=1)
    
    # restrict the bond features to real atoms
    # [ V, V, 12 ]
    pairs = tf.gather(tf.gather(datapoint['pairs'], atom_indices, axis=0), atom_indices, axis=1)
    
    # pair_mask is non-zero only for real bonds
    # [ E, 2 ]
    bond_indices = tf.where(pair_mask)
    
    # only keep features of real bonds
    # [ E, 12 ]
    bond_features = tf.gather_nd(pairs, bond_indices)
    
    # separate sources and targets for each bond
    # [ E, ]
    sources, targets = tf.unstack(tf.transpose(bond_indices))

    # active is [1, 0] for non-toxic molecules, [0, 1] for toxic molecules
    # [ ]
    toxicity = tf.argmax(datapoint['active'])
    
    # the molecule_id is included for reference
    # [ ]
    molecule_id = datapoint['molecule_id']

    # create a GraphTensor from all of the above
    atom = tfgnn.NodeSet.from_fields(features={'atom_features': atom_features},
                                     sizes=tf.shape(atom_indices))
    
    atom_adjacency = tfgnn.Adjacency.from_indices(source=('atom', tf.cast(sources, dtype=tf.int32)),
                                                  target=('atom', tf.cast(targets, dtype=tf.int32)))
    
    bond = tfgnn.EdgeSet.from_fields(features={'bond_features': bond_features},
                                     sizes=tf.shape(sources),
                                     adjacency=atom_adjacency)
    
    context = tfgnn.Context.from_fields(features={'toxicity': [toxicity], 'molecule_id': [molecule_id]})
    
    return tfgnn.GraphTensor.from_pieces(node_sets={'atom': atom}, edge_sets={'bond': bond}, context=context)

我们现在可以将此函数映射到数据集上，让它们流式传输 GraphTensor 对象：

In [8]:
train_dataset = dataset_splits['train'].map(make_graph_tensor)

In [9]:
graph_tensor = next(iter(train_dataset))
graph_tensor

GraphTensor(
  context=Context(features={'toxicity': <tf.Tensor: shape=(1,), dtype=tf.int64>, 'molecule_id': <tf.Tensor: shape=(1,), dtype=tf.string>}, sizes=[1], shape=(), indices_dtype=tf.int32),
  node_set_names=['atom'],
  edge_set_names=['bond'])

并检查由此产生的 GraphTensor 是否与我们之前定义的 GraphTensorSpec 兼容：

In [10]:
graph_spec.is_compatible_with(graph_tensor)

True

但是，为了避免多次处理数据（这会减慢所有输入管道的速度），首先将所有数据转储到 TFRecord 文件中会很方便。 稍后我们可以轻松加载这些数据集，而不是我们映射 make_graph_tensor 函数的原始 TF-DS 数据集。

注意：下面的 create_tfrecords 方法运行良好，相当通用，可以立即重用于其他小型应用程序。 然而，对于大规模数据集，使用 tf.data.Dataset.cache 或 tf.data.Dataset.snapshot 的替代方法会更好，因为它们将允许更多优化，例如 分片。

In [11]:
def create_tfrecords(dataset_splits, dataset_info):
    """
    Dump all splits of the given dataset to TFRecord files.
    """
    for split_name, dataset in dataset_splits.items():
        filename = f'data/{dataset_info.name}-{split_name}.tfrecord'
        print(f'creating {filename}...')
        
        # convert all datapoints to GraphTensor
        dataset = dataset.map(make_graph_tensor, num_parallel_calls=tf.data.AUTOTUNE)
        
        # serialize to TFRecord files
        with tf.io.TFRecordWriter(filename) as writer:
            for graph_tensor in tqdm(iter(dataset), total=dataset_info.splits[split_name].num_examples):
                example = tfgnn.write_example(graph_tensor)
                writer.write(example.SerializeToString())

In [12]:
create_tfrecords(dataset_splits, dataset_info)

creating data/cardiotox-train.tfrecord...


100%|██████████| 6523/6523 [00:27<00:00, 238.27it/s]


creating data/cardiotox-validation.tfrecord...


100%|██████████| 1631/1631 [00:06<00:00, 240.76it/s]


creating data/cardiotox-test.tfrecord...


100%|██████████| 839/839 [00:03<00:00, 248.63it/s]


creating data/cardiotox-test2.tfrecord...


100%|██████████| 177/177 [00:00<00:00, 233.50it/s]


最后，我们可以使用 TFRecordDatasetProvider 类创建符合 DatasetProvider 的对象，该对象读取这些 TFRecord 文件并通过其 get_dataset 方法提供 tf.data.Dataset 对象供我们使用：

In [13]:
train_dataset_provider = runner.TFRecordDatasetProvider(file_pattern='data/cardiotox-train.tfrecord')
valid_dataset_provider = runner.TFRecordDatasetProvider(file_pattern='data/cardiotox-validation.tfrecord')
test1_dataset_provider = runner.TFRecordDatasetProvider(file_pattern='data/cardiotox-test.tfrecord')
test2_dataset_provider = runner.TFRecordDatasetProvider(file_pattern='data/cardiotox-test2.tfrecord')

## DatasetProvider 协议

上面定义的每个 DatasetProvider 通常都会生成一个序列化 GraphTensor 对象的数据集，我们需要在检查之前对其进行解析。 这里提到这一点仅供参考：编排器将在实际训练过程中透明地处理这一点。

为了获取数据集，我们需要提供输入上下文：

In [14]:
train_dataset = train_dataset_provider.get_dataset(context=tf.distribute.InputContext())

然后，我们将 tfgnn.parse_single_example 映射到该数据集，为我们的图指定适当的 GraphTensorSpec：

In [15]:
train_dataset = train_dataset.map(lambda serialized: tfgnn.parse_single_example(serialized=serialized, spec=graph_spec))

然后我们可以像以前一样流式传输 GraphTensor 对象

In [16]:
graph_tensor = next(iter(train_dataset))
graph_tensor

GraphTensor(
  context=Context(features={'molecule_id': <tf.Tensor: shape=(1,), dtype=tf.string>, 'toxicity': <tf.Tensor: shape=(1,), dtype=tf.int64>}, sizes=[1], shape=(), indices_dtype=tf.int32),
  node_set_names=['atom'],
  edge_set_names=['bond'])

## 数据核查

节点和边特征并不是特别说明性的，但是如果需要的话我们仍然可以直接访问它们。 首先，请注意该特定分子具有以下数字 V=|V| 原子数：  


In [17]:
graph_tensor.node_sets['atom'].sizes

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([33], dtype=int32)>

它们的特征被收集在形状 (V, 27) 的张量中，我们可以像这样访问它：

类似地，数字E=|E| 分子中的键为：

In [18]:
graph_tensor.node_sets['atom']['atom_features']

<tf.Tensor: shape=(33, 27), dtype=float32, numpy=
array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   

In [19]:
graph_tensor.edge_sets['bond'].sizes

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([68], dtype=int32)>

它们的特征被收集在形状为 (E, 12) 的张量中，我们可以像这样访问它：

In [20]:
graph_tensor.edge_sets['bond']['bond_features']

<tf.Tensor: shape=(68, 12), dtype=float32, numpy=
array([[1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
       [1., 0., 0., 0., 1., 1., 1., 1.

然后，边缘端点的 id 存储在几个形状为 (E,) 的张量中

In [21]:
graph_tensor.edge_sets['bond'].adjacency.source

<tf.Tensor: shape=(68,), dtype=int32, numpy=
array([ 0,  1,  1,  1,  2,  2,  2,  3,  3,  4,  4,  5,  5,  5,  6,  7,  7,
        8,  8,  9,  9, 10, 10, 10, 11, 12, 12, 13, 13, 14, 14, 15, 15, 15,
       16, 17, 17, 18, 18, 19, 19, 20, 20, 20, 21, 22, 23, 23, 23, 24, 25,
       25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 30, 31, 31, 31, 32],
      dtype=int32)>

In [22]:
graph_tensor.edge_sets['bond'].adjacency.target

<tf.Tensor: shape=(68,), dtype=int32, numpy=
array([ 1,  0,  2, 31,  1,  3, 23,  2,  4,  3,  5,  4,  6,  7,  5,  5,  8,
        7,  9,  8, 10,  9, 11, 12, 10, 10, 13, 12, 14, 13, 15, 14, 16, 17,
       15, 15, 18, 17, 19, 18, 20, 19, 21, 22, 20, 20,  2, 24, 25, 23, 23,
       26, 30, 25, 27, 26, 28, 27, 29, 28, 30, 25, 29, 31,  1, 30, 32, 31],
      dtype=int32)>

最后，有关图的全局信息由其上下文提供

In [23]:
graph_tensor.context['toxicity']

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>

In [24]:
graph_tensor.context['molecule_id']

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'CC1=C(C/C=C(\\C)CCC[C@H](C)CCC[C@H](C)CCCC(C)C)C(=O)c2ccccc2C1=O'],
      dtype=object)>

有了所有这些，我们可以编写以下辅助函数来可视化图表：

In [26]:
def draw_molecule(graph_tensor):
    """
    Plot the `GraphTensor` representation of a molecule.
    """
    (molecule_id,) = graph_tensor.context['molecule_id'].numpy()
    (toxicity,) = graph_tensor.context['toxicity'].numpy()

    sources = graph_tensor.edge_sets['bond'].adjacency.source.numpy()
    targets = graph_tensor.edge_sets['bond'].adjacency.target.numpy()

    pgvGraph = pgv.AGraph()
    pgvGraph.graph_attr['label'] = f'toxicity = {toxicity}\n\nmolecule_id = {molecule_id.decode()}'

    for edge in zip(sources, targets):
        pgvGraph.add_edge(edge)

    return Image(pgvGraph.draw(format='png', prog='dot'))

<img src="gnn.png" width="300"/>

## GraphTensor 批量化

GraphTensor 数据集可以像往常一样进行批处理，从而产生产生更高等级 GraphTensor 对象的新数据集：

In [31]:
batch_size = 64
batched_train_dataset = train_dataset.batch(batch_size)

In [32]:
graph_tensor_batch = next(iter(batched_train_dataset))
graph_tensor_batch.rank

1

生成的 GraphTensor 现在包含 tf.RaggedTensor 形式的特征，因为不同的图可以具有不同数量的节点和边：

In [33]:
graph_tensor_batch.node_sets['atom']['atom_features'].shape

TensorShape([64, None, 27])

In [34]:
graph_tensor_batch.edge_sets['bond']['bond_features'].shape

TensorShape([64, None, 12])

其中形状现在对应于 (batch_size, V, 27) 和 (batch_size, E, 12)。

然而，TF-GNN 中的所有层都期望标量图作为其输入，因此在实际使用一批图之前，我们应该始终将批次中的不同图“合并”为具有多个断开连接的组件的单个图（其中 TF-GNN 自动 跟踪）：

In [35]:
scalar_graph_tensor = graph_tensor_batch.merge_batch_to_components()
scalar_graph_tensor.rank

0

In [36]:
scalar_graph_tensor.node_sets['atom']['atom_features'].shape

TensorShape([1562, 27])

In [37]:
scalar_graph_tensor.edge_sets['bond']['bond_features'].shape

TensorShape([3370, 12])

然而，我们应该注意到，编排器将再次透明地为我们处理批处理和合并组件，因此只要我们不自定义训练例程，我们就不必担心这一点。

## 简单的MPNN模型

GNN 的常见架构由一个初始层组成，该初始层对图特征进行预处理，通常为节点和/或边生成隐藏状态，后面是一层或多层消息传递工作，如简介中所述。 本节的目标是定义一个 vanilla_mpnn_model 函数，可用于从以下位置创建此类简单的 GNN：

1. 执行预处理的初始层
1. 一个层堆叠多个消息传递层

## 初始化图

对于第一个任务，我们将使用 tfgnn.keras.layers.MapFeatures 层通过密集层传递原子和键的各自特征，为原子和键创建隐藏状态向量。 由此产生的隐藏状态将具有维度hidden_size，对应于dV和dE简介的符号
  。

以下辅助函数将为给定的超参数创建一个初始 MapFeatures 图层：

1. hidden_size：隐藏尺寸 dV 和 dE
1.  
激活：密集层的激活

In [38]:
def get_initial_map_features(hidden_size, activation='relu'):
    """
    Initial pre-processing layer for a GNN (use as a class constructor).
    """
    def node_sets_fn(node_set, node_set_name):
        if node_set_name == 'atom':
            return tf.keras.layers.Dense(units=hidden_size, activation=activation)(node_set['atom_features'])
    
    def edge_sets_fn(edge_set, edge_set_name):
        if edge_set_name == 'bond':
            return tf.keras.layers.Dense(units=hidden_size, activation=activation)(edge_set['bond_features'])
    
    return tfgnn.keras.layers.MapFeatures(node_sets_fn=node_sets_fn,
                                          edge_sets_fn=edge_sets_fn,
                                          name='graph_embedding')

我们可以检查结果层是否用指定维度的隐藏状态替换了“atom_features”和“bond_features”

In [39]:
graph_embedding = get_initial_map_features(hidden_size=128)

In [40]:
embedded_graph = graph_embedding(scalar_graph_tensor)

In [41]:
embedded_graph.node_sets['atom'].features

{'hidden_state': <tf.Tensor: shape=(1562, 128), dtype=float32, numpy=
array([[0.        , 0.2920603 , 0.16877857, ..., 0.02003821, 0.00672814,
        0.04361148],
       [0.        , 0.        , 0.02534644, ..., 0.04332802, 0.        ,
        0.        ],
       [0.        , 0.        , 0.02534644, ..., 0.04332802, 0.        ,
        0.        ],
       ...,
       [0.        , 0.32096922, 0.20735013, ..., 0.        , 0.09112668,
        0.14522932],
       [0.        , 0.14401045, 0.        , ..., 0.19082573, 0.        ,
        0.19302632],
       [0.        , 0.        , 0.        , ..., 0.1429371 , 0.        ,
        0.22474754]], dtype=float32)>}

请注意，原子和键特征现在都命名为“hidden_​​state”；我们当然可以选择不同的名称，但保留默认的 tfgnn.HIDDEN_STATE 将使我们不必在后面指定功能名称。

In [42]:
embedded_graph.edge_sets['bond'].features

{'hidden_state': <tf.Tensor: shape=(3370, 128), dtype=float32, numpy=
array([[0.0056771 , 0.07958694, 0.        , ..., 0.21486597, 0.41878182,
        0.2994969 ],
       [0.0056771 , 0.07958694, 0.        , ..., 0.21486597, 0.41878182,
        0.2994969 ],
       [0.        , 0.        , 0.        , ..., 0.12333959, 0.16223049,
        0.2355265 ],
       ...,
       [0.17426147, 0.        , 0.        , ..., 0.137586  , 0.36240458,
        0.40004316],
       [0.17426147, 0.        , 0.        , ..., 0.137586  , 0.36240458,
        0.40004316],
       [0.17426147, 0.        , 0.        , ..., 0.137586  , 0.36240458,
        0.40004316]], dtype=float32)>}

## 消息传递层的堆叠

为了说明如何构建消息传递层堆栈，我们将使用 models.gat_v2 模块中提供的预构建图注意力 (GAT) [2] 层。 然后，我们定义一个消息传递神经网络（MPNN）层，连续应用这些具有超参数的层：

1. hidden_size：隐藏尺寸 dV 和 dE
 
1. hops：堆栈的层数

In [43]:
class MPNN(tf.keras.layers.Layer):
    """
    A basic stack of message-passing Graph Attention layers.
    """
    def __init__(self, hidden_size, hops, name='gat_mpnn', **kwargs):
        self.hidden_size = hidden_size
        self.hops = hops
        super().__init__(name=name, **kwargs)
        
        self.mp_layers = [self._mp_factory(name=f'message_passing_{i}') for i in range(hops)]
    
    def _mp_factory(self, name):
        return gat_v2.GATv2GraphUpdate(num_heads=1,
                                       per_head_channels=self.hidden_size,
                                       edge_set_name='bond',
                                       sender_edge_feature=tfgnn.HIDDEN_STATE,
                                       name=name)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'hidden_size': self.hidden_size,
            'hops': self.hops
        })
        return config
        
    def call(self, graph_tensor):
        for layer in self.mp_layers:
            graph_tensor = layer(graph_tensor)
        return graph_tensor

我们现在可以检查该层是否处理来自初始特征图的嵌入图：

In [44]:
mpnn = MPNN(hidden_size=128, hops=8)

In [45]:
hidden_graph = mpnn(embedded_graph)

In [46]:
hidden_graph.node_sets['atom'].features

{'hidden_state': <tf.Tensor: shape=(1562, 128), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00236014, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00310817, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>}

In [47]:
hidden_graph.edge_sets['bond'].features

{'hidden_state': <tf.Tensor: shape=(3370, 128), dtype=float32, numpy=
array([[0.0056771 , 0.07958694, 0.        , ..., 0.21486597, 0.41878182,
        0.2994969 ],
       [0.0056771 , 0.07958694, 0.        , ..., 0.21486597, 0.41878182,
        0.2994969 ],
       [0.        , 0.        , 0.        , ..., 0.12333959, 0.16223049,
        0.2355265 ],
       ...,
       [0.17426147, 0.        , 0.        , ..., 0.137586  , 0.36240458,
        0.40004316],
       [0.17426147, 0.        , 0.        , ..., 0.137586  , 0.36240458,
        0.40004316],
       [0.17426147, 0.        , 0.        , ..., 0.137586  , 0.36240458,
        0.40004316]], dtype=float32)>}

## 组建模型

我们现在准备将这两种成分组合到 tf.keras.Model 中，该模型采用代表分子的 GraphTensor 作为输入，并生成另一个具有所有原子隐藏状态的 GraphTensor 作为输出。 我们使用 Keras 的功能 API 定义一个 vanilla_mpnn_model 辅助函数，返回所需的 tf.keras.Model：

In [48]:
def vanilla_mpnn_model(graph_tensor_spec, init_states_fn, pass_messages_fn):
    """
    Chain an initialization layer and a message-passing stack to produce a `tf.keras.Model`.
    """
    graph_tensor = tf.keras.layers.Input(type_spec=graph_tensor_spec)
    embedded_graph = init_states_fn(graph_tensor)
    hidden_graph = pass_messages_fn(embedded_graph)
    return tf.keras.Model(inputs=graph_tensor, outputs=hidden_graph)

In [49]:
model = vanilla_mpnn_model(graph_tensor_spec=graph_spec,
                           init_states_fn=graph_embedding,
                           pass_messages_fn=mpnn)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [()]                      0         
                                                                 
 graph_embedding (MapFeature  ()                       5248      
 s)                                                              
                                                                 
 gat_mpnn (MPNN)             ()                        396288    
                                                                 
Total params: 401,536
Trainable params: 401,536
Non-trainable params: 0
_________________________________________________________________


为了以后方便起见，让我们将所有这些逻辑封装在一个函数中，我们可以使用它来获取固定超参数的模型构造函数。 按照惯例，返回的构造函数仅采用模型输入图的 GraphTensorSpec，并且为了更好地衡量，我们的构造函数还将添加一些 L2 通过 l2_coefficient 超参数进行正则化：

In [50]:
def get_model_creation_fn(hidden_size, hops, activation='relu', l2_coefficient=1e-3):
    """
    Return a model constructor for a given set of hyperparameters.
    """
    def model_creation_fn(graph_tensor_spec):
        initial_map_features = get_initial_map_features(hidden_size=hidden_size, activation=activation)
        mpnn = MPNN(hidden_size=hidden_size, hops=hops)
        
        model = vanilla_mpnn_model(graph_tensor_spec=graph_tensor_spec,
                                   init_states_fn=initial_map_features,
                                   pass_messages_fn=mpnn)
        model.add_loss(lambda: tf.reduce_sum([tf.keras.regularizers.l2(l2=l2_coefficient)(weight) for weight in model.trainable_weights]))
        return model
    return model_creation_fn

In [51]:
mpnn_creation_fn = get_model_creation_fn(hidden_size=128, hops=8)

In [52]:
model = mpnn_creation_fn(graph_spec)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [()]                      0         
                                                                 
 graph_embedding (MapFeature  ()                       5248      
 s)                                                              
                                                                 
 gat_mpnn (MPNN)             ()                        396288    
                                                                 
Total params: 401,536
Trainable params: 401,536
Non-trainable params: 0
_________________________________________________________________


## 图二分类

任务配置

有了可用的 GNN 模型，我们现在准备将其应用于手头的任务，即预测其毒性的分子二元分类。 这涉及：

1. 添加读出和预测头，根据 GNN 计算的特征计算每个类别的逻辑。
1. 定义要最小化的损失函数，在本例中应该是分类交叉熵损失。
1. 定义我们在训练和验证期间有兴趣测量的指标。  

编排器定义了任务协议来实现这些目标，并方便地提供了一个符合该协议的预实现的 GraphBinaryClassification 类。 虽然我们可以按原样使用它，但出于说明目的，我们将通过两种方式扩展其基本实现：

1. 我们将包括 AUROC 指标。
1. 我们将概括读出和预测头以包括隐藏层。  

首先，我们围绕 tf.keras.metrics.AUC 类定义一个简单的包装器，以使其适应我们的约定：

In [53]:
class AUROC(tf.keras.metrics.AUC):
    """
    AUROC metric computation for binary classification from logits.
    
    y_true: true labels, with shape (batch_size,)
    y_pred: predicted logits, with shape (batch_size, 2)
    """
    def update_state(self, y_true, y_pred, sample_weight=None):
        super().update_state(y_true, tf.math.softmax(y_pred, axis=-1)[:,1])

接下来，我们对 GraphBinaryClassification 任务进行子类化并重写其调整和度量方法：

In [54]:
class GraphBinaryClassification(runner.GraphBinaryClassification):
    """
    A GraphBinaryClassification task with a hidden layer in the prediction head, and additional metrics.
    """
    def __init__(self, hidden_dim, *args, **kwargs):
        self._hidden_dim = hidden_dim
        super().__init__(*args, **kwargs)
        
    def adapt(self, model):
        hidden_state = tfgnn.pool_nodes_to_context(model.output,
                                                   node_set_name=self._node_set_name,
                                                   reduce_type=self._reduce_type,
                                                   feature_name=self._state_name)
        
        hidden_state = tf.keras.layers.Dense(units=self._hidden_dim, activation='relu', name='hidden_layer')(hidden_state)
        
        logits = tf.keras.layers.Dense(units=self._units, name='logits')(hidden_state)
        
        return tf.keras.Model(inputs=model.inputs, outputs=logits)
    
    def metrics(self):
        return (*super().metrics(), AUROC(name='AUROC'))

要创建此类的实例，我们需要指定将用于聚合隐藏状态以进行预测的节点集（请记住，在我们的例子中只有一个“原子”）和类的数量（两个，用于有毒和非 -有毒），以及新的超参数hidden_dim：

In [55]:
task = GraphBinaryClassification(hidden_dim=256, node_set_name='atom', num_classes=2)

然后，该实例提供了我们训练所需的一切，即： 损失函数

In [56]:
task.losses()

(<keras.losses.SparseCategoricalCrossentropy at 0x7f1a1840e3a0>,)

指标：

In [57]:
task.metrics()

(<keras.metrics.SparseCategoricalAccuracy at 0x7f1a1840ed00>,
 <__main__.AUROC at 0x7f1a1840e040>)

一种将读出和预测头放置在 GNN 顶部的适应方法

In [58]:
classification_model = task.adapt(model)
classification_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [()]                 0           []                               
                                                                                                  
 graph_embedding (MapFeatures)  ()                   5248        ['input_4[0][0]']                
                                                                                                  
 gat_mpnn (MPNN)                ()                   396288      ['graph_embedding[0][0]']        
                                                                                                  
 input.node_sets (InstancePrope  {'atom': ()}        0           ['gat_mpnn[0][0]']               
 rty)                                                                                       

然后，生成的模型会为每个类生成 logits，并将 GraphTensor 作为输入：

In [59]:
classification_model(graph_tensor)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.2722813 ,  0.09090953]], dtype=float32)>

预处理器方法，可用于在到达 GNN 之前对图进行预处理，但此处仍不使用

In [60]:
task.preprocessors()

()

## 训练

我们现在准备好训练模型了。首先，我们创建一个 KerasTrainer 实例，它利用 Keras 的 fit 方法实现协调器的 Trainer 协议：

In [61]:
trainer = runner.KerasTrainer(strategy=tf.distribute.get_strategy(), model_dir='model')

接下来，我们定义一个符合 GraphTensorProcessorFn 协议的简单函数，该函数从 GraphTensor 对象中提取标签，以便在监督训练期间使用（该函数将映射到数据集，然后传递到 tf.keras.Model.fit 方法）：

In [62]:
def extract_labels(graph_tensor):
    """
    Extract the toxicity class label from the `GraphTensor` representation of a molecule.
    Return a pair compatible with the `tf.keras.Model.fit` method.
    """
    return graph_tensor, graph_tensor.context['toxicity']

最后，我们可以把所有东西放在一起，一边喝咖啡，一边观看一些进度条的移动:-)

In [63]:
runner.run(
    train_ds_provider=train_dataset_provider,
    valid_ds_provider=valid_dataset_provider,
    feature_processors=[extract_labels],
    model_fn=get_model_creation_fn(hidden_size=128, hops=8),
    task=task,
    trainer=trainer,
    epochs=20,
    optimizer_fn=tf.keras.optimizers.Adam,
    gtspec=graph_spec,
    global_batch_size=128
)

Epoch 1/20


/home/heng/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_7/gat_mpnn/message_passing_7/node_set_update_7/gat_v2_conv/Reshape_3:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_7/gat_mpnn/message_passing_7/node_set_update_7/gat_v2_conv/Reshape_2:0", shape=(None, 1, 1), dtype=float32), dense_shape=Tensor("gradient_tape/model_7/gat_mpnn/message_passing_7/node_set_update_7/gat_v2_conv/Cast:0", shape=(3,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/home/heng/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_7/gat_mpnn/message_passing_7/node_set_update_7/gat_v2_conv/Reshape_6:0", shape=(None,), dtype=int32), values=Tensor("

50/50 [==============================] - 7s 58ms/step - loss: 0.5513 - sparse_categorical_accuracy: 0.7359 - sparse_categorical_crossentropy: 0.5513 - AUROC: 0.6318 - val_loss: 0.5147 - val_sparse_categorical_accuracy: 0.7305 - val_sparse_categorical_crossentropy: 0.5147 - val_AUROC: 0.7290
Epoch 2/20
50/50 [==============================] - 2s 40ms/step - loss: 0.4745 - sparse_categorical_accuracy: 0.7588 - sparse_categorical_crossentropy: 0.4745 - AUROC: 0.7798 - val_loss: 0.4755 - val_sparse_categorical_accuracy: 0.7565 - val_sparse_categorical_crossentropy: 0.4755 - val_AUROC: 0.7876
Epoch 3/20
50/50 [==============================] - 2s 40ms/step - loss: 0.4467 - sparse_categorical_accuracy: 0.7841 - sparse_categorical_crossentropy: 0.4467 - AUROC: 0.8148 - val_loss: 0.4552 - val_sparse_categorical_accuracy: 0.7786 - val_sparse_categorical_crossentropy: 0.4552 - val_AUROC: 0.8117
Epoch 4/20
50/50 [==============================] - 2s 40ms/step - loss: 0.4255 - sparse_categorical_a

2023-10-29 22:06:18.770933: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/home/heng/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/saved_model/nested_structure_coder.py:522: UserWarning: Encoding a StructuredValue with type tensorflow_gnn.GraphTensorSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
/home/heng/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/saved_model/nested_structure_coder.py:522: UserWarning: Encoding a StructuredValue with type tensorflow_gnn.ContextSpec.v2; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
/home/heng/anaconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/saved_model/nested_structure_co

## 指标可视化

可视化训练和验证期间收集的各种指标的一种直接方法是使用 TensorBoard。理想情况下，以下魔法应该起作用：

In [71]:
!kill 401550

In [72]:
%load_ext tensorboard
%tensorboard --logdir model --bind_all
%reload_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<img src="gnn2.png" width="400"/>

准确率达到87%左右
<img src="gnn3.png" width="400"/>

损失逐渐减少，直到我们开始过度拟合（橙色线是训练，蓝色线是验证）：
<img src="gnn4.png" width="400"/>

## 结论

在本笔记本中，我们了解了如何使用 TF-GNN 以端到端的方式训练用于图二元分类的 GNN 模型。 运行协调器的最后一个单元汇集了我们在此过程中引入的所有元素，即：

1. 构造的 DatasetProvider 兼容对象 train_dataset_provider 和 valid_dataset_provider 用于提供数据
1. 内置的模型构造函数 get_model_creation_fn 与 组件一起组装 GNN
1. 定义的 GraphBinaryClassification 任务指定读出和预测头，以及损失和指标。
1. 创建的 KerasTrainer 和目标特征提取器用于监督训练  

虽然从这样一个小例子中可能不会立即明显看出，但 TF-GNN 在每个步骤中都提供了帮助，不仅提供了我们需要在图上执行的底层操作，而且还提供了许多有用的协议和辅助函数来处理大部分样板代码 否则我们会有要求。 将它们与协调器一起使用意味着所有组件都可以轻松扩展和/或替换。 此外，它至少在原则上允许我们轻松地独立缩放各个移动部件，而不会产生不必要的痛苦。 例如，在训练器中引入一个重要的策略，我们可以将训练分布在多个 GPU 上，或者最终分布在 TPU 上，同时还可以通过传递到 DatasetProvider 的 InputContext 并行化我们的输入管道。

我们获得的药物心脏毒性数据集的结果很好，但并不令人印象深刻。 考虑到我们实现的非常简单的基于 GAT 的模型以及我们没有进行超参数优化或有原则的架构选择这一事实，这是可以预料到的。 为了进行比较，其中我们看到我们的AUROC结果与那里考虑的GNN基线基本一致：

<img src="gnn5.png" width="400"/>

## 参考
1. [Introduction to TF-GNN](https://www.kaggle.com/code/fidels/introduction-to-tf-gnn/notebook)
2. [Graph Neural Networks: Graph Classification](https://blog.dataiku.com/graph-neural-networks-part-three)
3. [A Gentle Introduction to Graph Neural Networks](https://distill.pub/2021/gnn-intro/)
4. [Graph Representation Learning Book](https://www.cs.mcgill.ca/~wlh/grl_book/)
5. [TensorFlow GNN guide](https://github.com/tensorflow/gnn/blob/main/tensorflow_gnn/docs/guide/intro.md)